<a href="https://colab.research.google.com/github/SaadUSheikh/SafetyandclassificationwithSParktensorpytorch/blob/main/sparkforsafet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install PySpark if not already installed
!pip install pyspark


In [13]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder.appName("VictoriaRoadCrashAnalysis").getOrCreate()

# Load the dataset
file_path = 'Victoria_Road_Crash_Dataset_core.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show first few rows
print(df.columns)

df.show(5)


['ACCIDENT_NO', 'ACCIDENT_DATE', 'ACCIDENT_TIME', 'ACCIDENT_TYPE', 'DAY_OF_WEEK', 'DCA_CODE', 'DCA_CODE_DESCRIPTION', 'LIGHT_CONDITION', 'POLICE_ATTEND', 'ROAD_GEOMETRY', 'SEVERITY', 'SPEED_ZONE', 'RUN_OFFROAD', 'ROAD_NAME', 'ROAD_TYPE', 'ROAD_ROUTE_1', 'LGA_NAME', 'DTP_REGION', 'LATITUDE', 'LONGITUDE', 'VICGRID_X', 'VICGRID_Y', 'TOTAL_PERSONS', 'INJ_OR_FATAL', 'FATALITY', 'SERIOUSINJURY', 'OTHERINJURY', 'NONINJURED', 'MALES', 'FEMALES', 'BICYCLIST', 'PASSENGER', 'DRIVER', 'PEDESTRIAN', 'PILLION', 'MOTORCYCLIST', 'UNKNOWN', 'PED_CYCLIST_5_12', 'PED_CYCLIST_13_18', 'OLD_PED_65_AND_OVER', 'OLD_DRIVER_75_AND_OVER', 'YOUNG_DRIVER_18_25', 'NO_OF_VEHICLES', 'HEAVYVEHICLE', 'PASSENGERVEHICLE', 'MOTORCYCLE', 'PT_VEHICLE', 'DEG_URBAN_NAME', 'SRNS', 'RMA', 'DIVIDED', 'STAT_DIV_NAME']
+------------+-------------+-------------+--------------------+-----------+--------+--------------------+--------------------+-------------+-------------------+--------------------+----------+-----------+-----------

In [14]:
# Check schema to understand data types
df.printSchema()

# Summary statistics
df.describe().show()

# Count rows
print(f"Total rows: {df.count()}")


root
 |-- ACCIDENT_NO: string (nullable = true)
 |-- ACCIDENT_DATE: integer (nullable = true)
 |-- ACCIDENT_TIME: integer (nullable = true)
 |-- ACCIDENT_TYPE: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- DCA_CODE: integer (nullable = true)
 |-- DCA_CODE_DESCRIPTION: string (nullable = true)
 |-- LIGHT_CONDITION: string (nullable = true)
 |-- POLICE_ATTEND: string (nullable = true)
 |-- ROAD_GEOMETRY: string (nullable = true)
 |-- SEVERITY: string (nullable = true)
 |-- SPEED_ZONE: string (nullable = true)
 |-- RUN_OFFROAD: string (nullable = true)
 |-- ROAD_NAME: string (nullable = true)
 |-- ROAD_TYPE: string (nullable = true)
 |-- ROAD_ROUTE_1: integer (nullable = true)
 |-- LGA_NAME: string (nullable = true)
 |-- DTP_REGION: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- VICGRID_X: double (nullable = true)
 |-- VICGRID_Y: double (nullable = true)
 |-- TOTAL_PERSONS: integer (nullable = true)

In [16]:
# Categorical columns to index
categorical_columns = [
    "ACCIDENT_TYPE",
    "DAY_OF_WEEK",
    "LIGHT_CONDITION",
    "POLICE_ATTEND",
    "ROAD_GEOMETRY",
    "ROAD_TYPE"
]

# StringIndexer for categorical columns
indexers = [StringIndexer(inputCol=col, outputCol=col + "_Index", handleInvalid='skip') for col in categorical_columns]

# Numeric columns (you can add more based on your use-case)
numeric_columns = ["TOTAL_PERSONS", "INJ_OR_FATAL", "FATALITY", "SERIOUSINJURY", "OTHERINJURY"]

# All features to be combined
feature_cols = [col + "_Index" for col in categorical_columns] + numeric_columns

# Assemble feature vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")


In [18]:
# Drop rows with nulls if needed
df = df.dropna(subset=["SEVERITY"])

# Split dataset
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Fit the model
model = pipeline.fit(train_data)


In [19]:
# Predict on test data
predictions = model.transform(test_data)

# Show some predictions
predictions.select("features", "label", "prediction", "probability").show(10)

# Evaluate the model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Model Accuracy: {accuracy:.2f}")


+--------------------+-----+----------+--------------------+
|            features|label|prediction|         probability|
+--------------------+-----+----------+--------------------+
|(11,[1,2,4,6,7,9]...|  1.0|       1.0|[2.55895007498426...|
|[2.0,6.0,1.0,1.0,...|  0.0|       0.0|[0.99978436886140...|
|(11,[1,4,6,7,9],[...|  1.0|       1.0|[2.19367416512849...|
|(11,[0,1,4,6,7,9]...|  1.0|       1.0|[3.22932762349599...|
|(11,[0,1,2,6,7,10...|  0.0|       0.0|[0.99963532540018...|
|(11,[1,3,4,6,7,9]...|  1.0|       1.0|[5.13682543236677...|
|(11,[0,1,5,6,7,9]...|  1.0|       1.0|[2.54052841074104...|
|(11,[1,6,7,9],[5....|  1.0|       1.0|[2.27697264375122...|
|[1.0,5.0,0.0,0.0,...|  0.0|       0.0|[0.99952925216154...|
|(11,[1,4,6,7,9],[...|  1.0|       1.0|[1.96413926838400...|
+--------------------+-----+----------+--------------------+
only showing top 10 rows

Model Accuracy: 1.00
